In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [2]:
titanic = sns.load_dataset("titanic")
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
model1 = smf.logit(formula='survived ~ 1', data=titanic).fit();
model1.params

Optimization terminated successfully.
         Current function value: 0.665912
         Iterations 4


Intercept   -0.473288
dtype: float64

In [4]:
cross_tab = pd.DataFrame({
    'count': titanic['survived'].value_counts(),
    'percentage': titanic['survived'].value_counts(normalize=True)
    })

round(cross_tab,2)

,count,percentage
survived,,
0,549,0.62
1,342,0.38


In [5]:
model3 = smf.logit(formula='survived ~ fare', data=titanic).fit()
model3.params

Optimization terminated successfully.
         Current function value: 0.627143
         Iterations 6


Intercept   -0.941330
fare         0.015197
dtype: float64

In [6]:
model2 = smf.logit(formula='survived ~ C(pclass)', data=titanic).fit()
model2.params

Optimization terminated successfully.
         Current function value: 0.607805
         Iterations 5


Intercept         0.530628
C(pclass)[T.2]   -0.639431
C(pclass)[T.3]   -1.670399
dtype: float64

In [7]:
model4 = smf.logit(formula='survived ~ fare + C(sex) + age', data=titanic).fit()
model4.summary()

Optimization terminated successfully.
         Current function value: 0.501450
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      710
Method:                           MLE   Df Model:                            3
Date:                Tue, 28 Oct 2025   Pseudo R-squ.:                  0.2576
Time:                        14:02:52   Log-Likelihood:                -358.04
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 1.419e-53
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.9348      0.239      3.910      0.000       0.466       1.403
C(sex)[T.male]    -2.3476      0.190    -12.359      0.000      -2.720      -1.975
fare               0.0128      0.003      4.738      0.000       0.007       0.018
age               -0.0106      0.006     -1.627      0.104      -0.023       0.002
==================================================================================
"""

In [8]:
mpg = sns.load_dataset('mpg').dropna().drop(columns=['origin', 'name', 'displacement'])
mpg.corr().style.background_gradient(cmap='coolwarm')

,mpg,cylinders,horsepower,weight,acceleration,model_year
mpg,1.000000,-0.777618,-0.778427,-0.832244,0.423329,0.580541
cylinders,-0.777618,1.000000,0.842983,0.897527,-0.504683,-0.345647
horsepower,-0.778427,0.842983,1.000000,0.864538,-0.689196,-0.416361
weight,-0.832244,0.897527,0.864538,1.000000,-0.416839,-0.309120
acceleration,0.423329,-0.504683,-0.689196,-0.416839,1.000000,0.290316
model_year,0.580541,-0.345647,-0.416361,-0.309120,0.290316,1.000000


In [9]:
mpg['lin_comb'] = 10 * mpg['cylinders'] - 0.3 * mpg['horsepower']
mpg.head(3)

,mpg,cylinders,horsepower,weight,acceleration,model_year,lin_comb
0,18.0,8,130.0,3504,12.0,70,41.0
1,15.0,8,165.0,3693,11.5,70,30.5
2,18.0,8,150.0,3436,11.0,70,35.0


In [11]:
# Matrix is not full-rank!
print(mpg.shape)
np.linalg.matrix_rank(mpg)

(392, 7)


np.int64(6)

In [13]:
smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().params

Intercept     528.876711
cylinders       3.375029
horsepower     16.840512
lin_comb       28.698140
dtype: float64

In [15]:
# Now, change just a bit one single observation by 1% just on one feature
mpg.loc[0,'horsepower'] = mpg.loc[0,'horsepower']*1.01
smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().params

Intercept      524.838981
cylinders     5954.582173
horsepower    -161.691947
lin_comb      -566.220542
dtype: float64

In [17]:
# Statsmodels gives us a clear WARNING [2]
# Summary table also reads 'Covariance Type: nonrobust'
smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 weight   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     712.9
Date:                Tue, 28 Oct 2025   Prob (F-statistic):          1.99e-157
Time:                        14:16:24   Log-Likelihood:                -2832.3
No. Observations:                 392   AIC:                             5673.
Df Residuals:                     388   BIC:                             5689.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    524.8390     56.865      9.230      0.000     413.038     636.640
cylinders   5954.5822   4286.694      1.389      0.166   -2473.473    1.44e+04
horsepower  -161.6919    128.597     -1.257      0.209    -414.526      91.142
lin_comb    -566.2205    428.515     -1.321      0.187   -1408.722     276.281
==============================================================================
Omnibus:                       12.222   Durbin-Watson:                   1.199
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               24.236
Skew:                           0.069   Prob(JB):                     5.46e-06
Kurtosis:                       4.210   Cond. No.                     2.91e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.91e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
mpg['lin_comb'] = mpg['lin_comb'] +  0.05 * np.random.rand(mpg.shape[0])
np.linalg.matrix_rank(mpg)

np.int64(7)

In [20]:
smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().params

Intercept      543.028388
cylinders     7563.981263
horsepower    -209.978602
lin_comb      -727.148435
dtype: float64

In [22]:
# Again, change just a bit one single observation in the dataset and check the OLS results
mpg.loc[0,'horsepower'] = mpg.loc[0,'horsepower']*0.8
smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().params

Intercept     524.875037
cylinders     261.948804
horsepower      9.088382
lin_comb        3.030824
dtype: float64

In [24]:
# Correlation matrix is not sufficient to detect soft or event strict multicollinearity
mpg.corr().style.background_gradient(cmap='coolwarm')

,mpg,cylinders,horsepower,weight,acceleration,model_year,lin_comb
mpg,1.000000,-0.777618,-0.776518,-0.832244,0.423329,0.580541,-0.445165
cylinders,-0.777618,1.000000,0.838737,0.897527,-0.504683,-0.345647,0.762598
horsepower,-0.776518,0.838737,1.000000,0.862883,-0.685508,-0.411591,0.289442
weight,-0.832244,0.897527,0.862883,1.000000,-0.416839,-0.309120,0.554605
acceleration,0.423329,-0.504683,-0.685508,-0.416839,1.000000,0.290316,-0.067757
model_year,0.580541,-0.345647,-0.411591,-0.309120,0.290316,1.000000,-0.113296
lin_comb,-0.445165,0.762598,0.289442,0.554605,-0.067757,-0.113296,1.000000


In [25]:
mpg_scaled = mpg.copy()

for feature in mpg_scaled.columns:
    mu = mpg[feature].mean()
    sigma = mpg[feature].std()
    mpg_scaled[feature] = mpg_scaled[feature].apply(lambda x: (x-mu)/sigma)

mpg_scaled

,mpg,cylinders,horsepower,weight,acceleration,model_year,lin_comb
0,-0.697747,1.482053,-0.506260,0.619748,-1.283618,-1.623241,1.836383
1,-1.082115,1.482053,1.575947,0.842258,-1.464852,-1.623241,0.740643
2,-0.697747,1.482053,1.186155,0.539692,-1.646086,-1.623241,1.208465
3,-0.953992,1.482053,1.186155,0.536160,-1.283618,-1.623241,1.211946
4,-0.825870,1.482053,0.926294,0.554997,-1.827320,-1.623241,1.525211
...,...,...,...,...,...,...,...
393,0.455359,-0.862911,-0.476956,-0.220842,0.021267,1.634321,-0.958006
394,2.633448,-0.862911,-1.360484,-0.997859,3.283479,1.634321,0.108020
395,1.095974,-0.862911,-0.528928,-0.803605,-1.428605,1.634321,-0.892681
396,0.583482,-0.862911,-0.658859,-0.415097,1.108671,1.634321,-0.734551


In [27]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
# compute VIF factor for feature index 0
vif(mpg_scaled.values, 0)

np.float64(5.237391734461867)

In [28]:
df = pd.DataFrame()

df["features"] = mpg_scaled.columns

df["vif_index"] = [vif(mpg_scaled.values, i) for i in range(mpg_scaled.shape[1])]

round(df.sort_values(by="vif_index", ascending = False),2)

,features,vif_index
1,cylinders,665.12
2,horsepower,291.99
6,lin_comb,213.05
3,weight,11.25
0,mpg,5.24
4,acceleration,2.61
5,model_year,1.91
